### Web Scraper to generate hisotrical weather data

In [1]:
import requests
import pprint
from pymongo import MongoClient
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import selenium
import numpy as np

First, clean up my data in order to identify which dates to scrape weather on <br/> Pulling historic weather data from Pheonix, AZ

In [2]:
'''
key items:

date
<div _ngcontent-app-root-c6="" class="date ng-star-inserted">1</div>

Weather description for the day
class = <div _ngcontent-app-root-c6="" class="phrase ng-star-inserted">mostly sunny</div>

Hi
<span _ngcontent-app-root-c6="" class="hi">80°</span>

Low
<span _ngcontent-app-root-c6="" class="low">61°</span>

Rain (in inches)
<span _ngcontent-app-root-c12="" class="wu-value wu-value-to">0.21</span>

FOCUS HERE
Phoenix, AZ
https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-4 
- date will be the changing variable
'''


'\nkey items:\n\ndate\n<div _ngcontent-app-root-c6="" class="date ng-star-inserted">1</div>\n\nWeather description for the day\nclass = <div _ngcontent-app-root-c6="" class="phrase ng-star-inserted">mostly sunny</div>\n\nHi\n<span _ngcontent-app-root-c6="" class="hi">80°</span>\n\nLow\n<span _ngcontent-app-root-c6="" class="low">61°</span>\n\nRain (in inches)\n<span _ngcontent-app-root-c12="" class="wu-value wu-value-to">0.21</span>\n\nFOCUS HERE\nPhoenix, AZ\nhttps://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-4 \n- date will be the changing variable\n'

### <center> Create a List of URL's to pull data from <center>

In [3]:
# Seems like I'll need to set up a web scraper for each year and have the month values change

url_lst_2019 = []

def web_scraper_links():
    url = 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1'
    for x in range(1,13):
        url_lst_2019.append(f"https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-{x}")
        
        
web_scraper_links()
# Above function worked just fine. Now have all 2019 weather data links

In [4]:
url_lst_2019

['https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-2',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-3',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-4',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-5',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-6',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-7',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-8',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-9',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-10',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-11',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-12']

In [5]:
# Selenium rough draft:
'''
# This starts up my automated browser
driver = webdriver.Safari()
# driver.get("https://www.wunderground.com/history/monthly/us/az/phoenix/KPHX/date/2019-1")
driver.get('https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1')
daily_data = driver.find_elements_by_class_name('calendar-day.current-month.history.ng-star-inserted')
for x in daily_data:
    print(
        x.find_element_by_class_name("date.ng-star-inserted").text, 
        x.find_element_by_class_name("phrase.ng-star-inserted").text,
        x.find_element_by_class_name("hi").text,
        x.find_element_by_class_name("low").text,
        x.find_element_by_class_name("wu-value.wu-value-to").text
         )
'''

'\n# This starts up my automated browser\ndriver = webdriver.Safari()\n# driver.get("https://www.wunderground.com/history/monthly/us/az/phoenix/KPHX/date/2019-1")\ndriver.get(\'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1\')\ndaily_data = driver.find_elements_by_class_name(\'calendar-day.current-month.history.ng-star-inserted\')\nfor x in daily_data:\n    print(\n        x.find_element_by_class_name("date.ng-star-inserted").text, \n        x.find_element_by_class_name("phrase.ng-star-inserted").text,\n        x.find_element_by_class_name("hi").text,\n        x.find_element_by_class_name("low").text,\n        x.find_element_by_class_name("wu-value.wu-value-to").text\n         )\n'

In [6]:
def get_weather_data(x):
    driver = webdriver.Safari()
    # change the hyperlink to a variable so I can call my function for certain months
    time.sleep(3)
    driver.get(x)
    time.sleep(3)
    daily_data = driver.find_elements_by_class_name('calendar-day.current-month.history.ng-star-inserted')
    day = []
    conditions = []
    for x in daily_data:
        date = x.find_element_by_class_name("date.ng-star-inserted").text
        day.append(date)
        condition = x.find_element_by_class_name("phrase.ng-star-inserted").text
        conditions.append(condition)
    return day, conditions

In [7]:
# Play-by-play in order to get the DF to what I need

In [8]:
day, conditions = get_weather_data('https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1')

In [9]:
lst = [day, conditions]

In [10]:
month_weather_data = pd.DataFrame(lst)

In [11]:
month_weather_data = month_weather_data.T

In [12]:
month_weather_data.columns = ['Day', 'Condition']

In [13]:
month_weather_data['Indicator'] = 0

In [14]:
month_weather_data['Indicator'] = month_weather_data['Condition'].apply(lambda x: 0 if x in ["cloudy", "mostly cloudy", "partly cloudy", "scattered showers", "thunderstorm"] else 1)

In [15]:
# SAVE TO CSV below!

In [16]:
month_weather_data

,Day,Condition,Indicator
0,1,mostly cloudy,0
1,2,mostly sunny,1
2,3,mostly sunny,1
3,4,mostly sunny,1
4,5,cloudy,0
5,6,cloudy,0
6,7,partly cloudy,0
7,8,mostly sunny,1
8,9,cloudy,0
9,10,cloudy,0


In [17]:
# month_weather_data.to_csv("jan.csv")

In [18]:
# month_weather_data.to_csv("feb.csv")

In [19]:
# month_weather_data.to_csv("mar.csv")

In [20]:
# month_weather_data.to_csv("apr.csv")

In [21]:
# month_weather_data.to_csv("may.csv")

In [22]:
# month_weather_data.to_csv("jun.csv")

In [23]:
# month_weather_data.to_csv("jul.csv")

In [24]:
# month_weather_data.to_csv("aug.csv")

In [25]:
# month_weather_data.to_csv("sep.csv")

In [26]:
# month_weather_data.to_csv("oct.csv")

In [27]:
# month_weather_data.to_csv("nov.csv")

In [28]:
# month_weather_data.to_csv("dec.csv")